In [1]:
import pandas as pd
df = pd.read_csv("main_feature_database.csv")

In [2]:
df.head()

,season,date,home,score,away,hPos,hGSPG,hGCPG,hCSPG,hPtsPG,...,hs2,as2,hst2,ast2,hSPG,hSTPG,aSPG,aSTPG,hS%,aS%
0,9,2000-08-19,charlton,4-0,man city,4.0,0.0,0.0,0.0,0.0,...,17.0,8.0,14.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0
1,9,2000-08-19,chelsea,4-2,west ham,5.0,0.0,0.0,0.0,0.0,...,17.0,12.0,10.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0
2,9,2000-08-19,coventry,1-3,middlesbrough,6.0,0.0,0.0,0.0,0.0,...,6.0,16.0,3.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0
3,9,2000-08-19,derby,2-2,southampton,7.0,0.0,0.0,0.0,0.0,...,6.0,13.0,4.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0
4,9,2000-08-19,leeds,2-0,everton,10.0,0.0,0.0,0.0,0.0,...,17.0,12.0,8.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
def get_points_per_game(last_n):
    total_points = 0
    scores = last_n['score'].tolist()
    sides = last_n['home'].tolist()
    for i in range(len(scores)):
        score = scores[i]
        side = sides[i]
        [home_goals, away_goals] = [int(x) for x in score.split("-")]
        if side:
            if home_goals > away_goals:
                total_points += 3
            elif home_goals == away_goals:
                total_points += 1
        else:
            if away_goals > home_goals:
                total_points += 3
            elif away_goals == home_goals:
                total_points += 1
    return round(total_points/len(scores),3)

def get_goal_difference(last_n):
    goal_difference = 0
    scores = last_n['score'].tolist()
    sides = last_n['home'].tolist()
    for i in range(len(scores)):
        score = scores[i]
        side = sides[i]
        [home_goals, away_goals] = [int(x) for x in score.split("-")]
        home_goal_diff = home_goals - away_goals
        if side:
            goal_difference += home_goal_diff
        else:
            goal_difference -= home_goal_diff
    return round(goal_difference/len(scores),3)

def get_form_values(row,df,homeBool):
    season = row['season']
    date = row['date']
    df_season = df.query("season == @season")
    df_relevant = df[pd.to_datetime(df.date) < pd.to_datetime(date)]
    
    if homeBool:
        team = row['home']
    else:
        team = row['away']
    
    games_when_home = df_relevant.query("home == @team")
    games_when_away = df_relevant.query("away == @team")
    
    data_when_home = games_when_home[['date','score']]
    data_when_away = games_when_away[['date','score']]
    
    data_when_home['home'] = [True]*len(data_when_home)
    data_when_away['home'] = [False]*len(data_when_away)
    
    all_relevant_games = pd.concat([data_when_home,data_when_away],axis=0)
    all_relevant_games.sort_values(by='date',inplace=True,axis=0)
    
    PTSPG_last3, PTSPG_last5, PTSPG_last7 = [0,0,0]
    GD_last3, GD_last5, GD_last7 = [0,0,0]
    
    if len(all_relevant_games) > 2:
        PTSPG_last3 = get_points_per_game(all_relevant_games.tail(3))
        GD_last3 = get_goal_difference(all_relevant_games.tail(3))
    if len(all_relevant_games) > 4:
        PTSPG_last5 = get_points_per_game(all_relevant_games.tail(5))
        GD_last5 = get_goal_difference(all_relevant_games.tail(5))
    if len(all_relevant_games) > 6:
        PTSPG_last7 = get_points_per_game(all_relevant_games.tail(7))
        GD_last7 = get_goal_difference(all_relevant_games.tail(7))
        
    print(date)
        
    return PTSPG_last3, PTSPG_last5, PTSPG_last7, GD_last3, GD_last5, GD_last7    

In [4]:
df['hPTSPG3'],df['hPTSPG5'],df['hPTSPG7'],df['hGD3'],df['hGD5'],df['hGD7'] = zip(*df.apply(get_form_values,args=(df,True),axis=1))

2000-08-19
2000-08-19
2000-08-19
2000-08-19
2000-08-19
2000-08-19
2000-08-19
2000-08-19
2000-08-19
2000-08-20
2000-08-21
2000-08-22
2000-08-22


/home/joe/.local/lib/python3.6/site-packages/ipykernel_launcher.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/joe/.local/lib/python3.6/site-packages/ipykernel_launcher.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


2000-08-22
2000-08-23
2000-08-23
2000-08-23
2000-08-23
2000-08-23
2000-08-26
2000-08-26
2000-08-26
2000-08-26
2000-08-26
2000-08-26
2000-08-26
2000-08-26
2000-08-26
2000-08-27
2000-09-05
2000-09-05
2000-09-05
2000-09-05
2000-09-06
2000-09-06
2000-09-06
2000-09-06
2000-09-06
2000-09-06
2000-09-09
2000-09-09
2000-09-09
2000-09-09
2000-09-09
2000-09-09
2000-09-09
2000-09-09
2000-09-10
2000-09-11
2000-09-16
2000-09-16
2000-09-16
2000-09-16
2000-09-16
2000-09-16
2000-09-16
2000-09-17
2000-09-17
2000-09-17
2000-09-23
2000-09-23
2000-09-23
2000-09-23
2000-09-23
2000-09-23
2000-09-23
2000-09-23
2000-09-23
2000-09-24
2000-09-30
2000-09-30
2000-09-30
2000-09-30
2000-09-30
2000-09-30
2000-09-30
2000-10-01
2000-10-01
2000-10-01
2000-10-14
2000-10-14
2000-10-14
2000-10-14
2000-10-14
2000-10-14
2000-10-14
2000-10-14
2000-10-15
2000-10-16
2000-10-21
2000-10-21
2000-10-21
2000-10-21
2000-10-21
2000-10-21
2000-10-21
2000-10-21
2000-10-22
2000-10-23
2000-10-28
2000-10-28
2000-10-28
2000-10-28
2000-10-28

2002-05-11
2002-08-17
2002-08-17
2002-08-17
2002-08-17
2002-08-17
2002-08-17
2002-08-17
2002-08-18
2002-08-18
2002-08-19
2002-08-23
2002-08-24
2002-08-24
2002-08-24
2002-08-24
2002-08-24
2002-08-24
2002-08-24
2002-08-24
2002-08-24
2002-08-27
2002-08-27
2002-08-28
2002-08-28
2002-08-28
2002-08-28
2002-08-28
2002-08-31
2002-08-31
2002-08-31
2002-08-31
2002-08-31
2002-08-31
2002-08-31
2002-09-01
2002-09-01
2002-09-02
2002-09-03
2002-09-10
2002-09-10
2002-09-11
2002-09-11
2002-09-11
2002-09-11
2002-09-11
2002-09-11
2002-09-11
2002-09-11
2002-09-14
2002-09-14
2002-09-14
2002-09-14
2002-09-14
2002-09-14
2002-09-14
2002-09-15
2002-09-15
2002-09-16
2002-09-21
2002-09-21
2002-09-21
2002-09-21
2002-09-21
2002-09-21
2002-09-21
2002-09-22
2002-09-22
2002-09-23
2002-09-28
2002-09-28
2002-09-28
2002-09-28
2002-09-28
2002-09-28
2002-09-28
2002-09-28
2002-09-28
2002-09-30
2002-10-05
2002-10-05
2002-10-05
2002-10-05
2002-10-06
2002-10-06
2002-10-06
2002-10-06
2002-10-06
2002-10-07
2002-10-19
2002-10-19

2004-05-08
2004-05-08
2004-05-09
2004-05-09
2004-05-12
2004-05-15
2004-05-15
2004-05-15
2004-05-15
2004-05-15
2004-05-15
2004-05-15
2004-05-15
2004-05-15
2004-05-15
2004-08-14
2004-08-14
2004-08-14
2004-08-14
2004-08-14
2004-08-14
2004-08-14
2004-08-14
2004-08-15
2004-08-15
2004-08-21
2004-08-21
2004-08-21
2004-08-21
2004-08-21
2004-08-21
2004-08-21
2004-08-21
2004-08-22
2004-08-22
2004-08-24
2004-08-24
2004-08-25
2004-08-25
2004-08-25
2004-08-25
2004-08-25
2004-08-25
2004-08-28
2004-08-28
2004-08-28
2004-08-28
2004-08-28
2004-08-28
2004-08-28
2004-08-28
2004-08-29
2004-08-30
2004-08-30
2004-09-11
2004-09-11
2004-09-11
2004-09-11
2004-09-11
2004-09-11
2004-09-11
2004-09-11
2004-09-12
2004-09-13
2004-09-18
2004-09-18
2004-09-18
2004-09-18
2004-09-18
2004-09-18
2004-09-19
2004-09-19
2004-09-19
2004-09-20
2004-09-25
2004-09-25
2004-09-25
2004-09-25
2004-09-25
2004-09-25
2004-09-25
2004-09-25
2004-09-26
2004-09-27
2004-10-02
2004-10-02
2004-10-02
2004-10-02
2004-10-02
2004-10-02
2004-10-03

...

2010-04-04
2010-04-04
2010-04-04
2010-04-10
2010-04-10
2010-04-11
2010-04-11
2010-04-11
2010-04-11
2010-04-13
2010-04-14
2010-04-14
2010-04-14
2010-04-17
2010-04-17
2010-04-17
2010-04-17
2010-04-17
2010-04-17
2010-04-17
2010-04-18
2010-04-18
2010-04-19
2010-04-21
2010-04-24
2010-04-24
2010-04-24
2010-04-24
2010-04-24
2010-04-24
2010-04-25
2010-04-25
2010-04-25
2010-04-25
2010-05-01
2010-05-01
2010-05-01
2010-05-01
2010-05-01
2010-05-02
2010-05-02
2010-05-02
2010-05-03
2010-05-03
2010-05-05
2010-05-05
2010-05-09
2010-05-09
2010-05-09
2010-05-09
2010-05-09
2010-05-09
2010-05-09
2010-05-09
2010-05-09
2010-05-09
2010-08-14
2010-08-14
2010-08-14
2010-08-14
2010-08-14
2010-08-14
2010-08-14
2010-08-14
2010-08-15
2010-08-16
2010-08-21
2010-08-21
2010-08-21
2010-08-21
2010-08-21
2010-08-21
2010-08-21
2010-08-22
2010-08-22
2010-08-23
2010-08-28
2010-08-28
2010-08-28
2010-08-28
2010-08-28
2010-08-28
2010-08-29
2010-08-29
2010-08-29
2010-08-29
2010-09-11
2010-09-11
2010-09-11
2010-09-11
2010-09-11

2012-04-07
2012-04-07
2012-04-07
2012-04-07
2012-04-07
2012-04-07
2012-04-07
2012-04-08
2012-04-08
2012-04-09
2012-04-09
2012-04-09
2012-04-09
2012-04-09
2012-04-10
2012-04-11
2012-04-11
2012-04-11
2012-04-11
2012-04-14
2012-04-14
2012-04-14
2012-04-14
2012-04-15
2012-04-16
2012-04-21
2012-04-21
2012-04-21
2012-04-21
2012-04-21
2012-04-21
2012-04-21
2012-04-22
2012-04-22
2012-04-22
2012-04-24
2012-04-28
2012-04-28
2012-04-28
2012-04-28
2012-04-28
2012-04-28
2012-04-28
2012-04-29
2012-04-29
2012-04-30
2012-05-01
2012-05-01
2012-05-02
2012-05-02
2012-05-05
2012-05-06
2012-05-06
2012-05-06
2012-05-06
2012-05-06
2012-05-06
2012-05-06
2012-05-07
2012-05-08
2012-05-13
2012-05-13
2012-05-13
2012-05-13
2012-05-13
2012-05-13
2012-05-13
2012-05-13
2012-05-13
2012-05-13
2012-08-18
2012-08-18
2012-08-18
2012-08-18
2012-08-18
2012-08-18
2012-08-18
2012-08-19
2012-08-19
2012-08-20
2012-08-22
2012-08-25
2012-08-25
2012-08-25
2012-08-25
2012-08-25
2012-08-25
2012-08-25
2012-08-26
2012-08-26
2012-09-01

2014-03-22
2014-03-22
2014-03-22
2014-03-22
2014-03-23
2014-03-23
2014-03-25
2014-03-25
2014-03-25
2014-03-26
2014-03-26
2014-03-29
2014-03-29
2014-03-29
2014-03-29
2014-03-29
2014-03-29
2014-03-29
2014-03-30
2014-03-30
2014-03-31
2014-04-05
2014-04-05
2014-04-05
2014-04-05
2014-04-05
2014-04-05
2014-04-05
2014-04-06
2014-04-06
2014-04-07
2014-04-12
2014-04-12
2014-04-12
2014-04-12
2014-04-12
2014-04-12
2014-04-13
2014-04-13
2014-04-15
2014-04-16
2014-04-16
2014-04-19
2014-04-19
2014-04-19
2014-04-19
2014-04-19
2014-04-19
2014-04-20
2014-04-20
2014-04-20
2014-04-21
2014-04-26
2014-04-26
2014-04-26
2014-04-26
2014-04-26
2014-04-26
2014-04-27
2014-04-27
2014-04-27
2014-04-28
2014-05-03
2014-05-03
2014-05-03
2014-05-03
2014-05-03
2014-05-03
2014-05-03
2014-05-04
2014-05-04
2014-05-05
2014-05-06
2014-05-07
2014-05-07
2014-05-11
2014-05-11
2014-05-11
2014-05-11
2014-05-11
2014-05-11
2014-05-11
2014-05-11
2014-05-11
2014-05-11
2014-08-16
2014-08-16
2014-08-16
2014-08-16
2014-08-16
2014-08-16

2016-03-05
2016-03-06
2016-03-06
2016-03-12
2016-03-12
2016-03-12
2016-03-13
2016-03-14
2016-03-19
2016-03-19
2016-03-19
2016-03-19
2016-03-19
2016-03-19
2016-03-20
2016-03-20
2016-03-20
2016-03-20
2016-04-02
2016-04-02
2016-04-02
2016-04-02
2016-04-02
2016-04-02
2016-04-02
2016-04-02
2016-04-03
2016-04-03
2016-04-09
2016-04-09
2016-04-09
2016-04-09
2016-04-09
2016-04-09
2016-04-09
2016-04-10
2016-04-10
2016-04-10
2016-04-13
2016-04-16
2016-04-16
2016-04-16
2016-04-16
2016-04-16
2016-04-16
2016-04-17
2016-04-17
2016-04-17
2016-04-18
2016-04-19
2016-04-20
2016-04-20
2016-04-20
2016-04-21
2016-04-23
2016-04-23
2016-04-23
2016-04-23
2016-04-24
2016-04-24
2016-04-25
2016-04-30
2016-04-30
2016-04-30
2016-04-30
2016-04-30
2016-04-30
2016-05-01
2016-05-01
2016-05-01
2016-05-02
2016-05-07
2016-05-07
2016-05-07
2016-05-07
2016-05-07
2016-05-07
2016-05-07
2016-05-08
2016-05-08
2016-05-08
2016-05-10
2016-05-11
2016-05-11
2016-05-11
2016-05-15
2016-05-15
2016-05-15
2016-05-15
2016-05-15
2016-05-15

2018-02-24
2018-02-24
2018-02-24
2018-02-24
2018-02-24
2018-02-25
2018-02-25
2018-03-01
2018-03-03
2018-03-03
2018-03-03
2018-03-03
2018-03-03
2018-03-03
2018-03-03
2018-03-04
2018-03-04
2018-03-05
2018-03-10
2018-03-10
2018-03-10
2018-03-10
2018-03-10
2018-03-10
2018-03-10
2018-03-11
2018-03-11
2018-03-12
2018-03-17
2018-03-17
2018-03-17
2018-03-17
2018-03-31
2018-03-31
2018-03-31
2018-03-31
2018-03-31
2018-03-31
2018-03-31
2018-03-31
2018-04-01
2018-04-01
2018-04-07
2018-04-07
2018-04-07
2018-04-07
2018-04-07
2018-04-07
2018-04-07
2018-04-07
2018-04-08
2018-04-08
2018-04-14
2018-04-14
2018-04-14
2018-04-14
2018-04-14
2018-04-14
2018-04-14
2018-04-15
2018-04-15
2018-04-16
2018-04-17
2018-04-18
2018-04-19
2018-04-19
2018-04-21
2018-04-21
2018-04-22
2018-04-22
2018-04-22
2018-04-23
2018-04-28
2018-04-28
2018-04-28
2018-04-28
2018-04-28
2018-04-28
2018-04-28
2018-04-29
2018-04-29
2018-04-30
2018-05-04
2018-05-05
2018-05-05
2018-05-05
2018-05-05
2018-05-05
2018-05-05
2018-05-06
2018-05-06

In [5]:
df['aPTSPG3'],df['aPTSPG5'],df['aPTSPG7'],df['aGD3'],df['aGD5'],df['aGD7'] = zip(*df.apply(get_form_values,args=(df,False),axis=1))

2000-08-19
2000-08-19
2000-08-19
2000-08-19
2000-08-19
2000-08-19
2000-08-19
2000-08-19
2000-08-19
2000-08-20
2000-08-21
2000-08-22


/home/joe/.local/lib/python3.6/site-packages/ipykernel_launcher.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


2000-08-22
2000-08-22
2000-08-23
2000-08-23
2000-08-23
2000-08-23
2000-08-23
2000-08-26
2000-08-26
2000-08-26


/home/joe/.local/lib/python3.6/site-packages/ipykernel_launcher.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


2000-08-26
2000-08-26
2000-08-26
2000-08-26
2000-08-26
2000-08-26
2000-08-27
2000-09-05
2000-09-05
2000-09-05
2000-09-05
2000-09-06
2000-09-06
2000-09-06
2000-09-06
2000-09-06
2000-09-06
2000-09-09
2000-09-09
2000-09-09
2000-09-09
2000-09-09
2000-09-09
2000-09-09
2000-09-09
2000-09-10
2000-09-11
2000-09-16
2000-09-16
2000-09-16
2000-09-16
2000-09-16
2000-09-16
2000-09-16
2000-09-17
2000-09-17
2000-09-17
2000-09-23
2000-09-23
2000-09-23
2000-09-23
2000-09-23
2000-09-23
2000-09-23
2000-09-23
2000-09-23
2000-09-24
2000-09-30
2000-09-30
2000-09-30
2000-09-30
2000-09-30
2000-09-30
2000-09-30
2000-10-01
2000-10-01
2000-10-01
2000-10-14
2000-10-14
2000-10-14
2000-10-14
2000-10-14
2000-10-14
2000-10-14
2000-10-14
2000-10-15
2000-10-16
2000-10-21
2000-10-21
2000-10-21
2000-10-21
2000-10-21
2000-10-21
2000-10-21
2000-10-21
2000-10-22
2000-10-23
2000-10-28
2000-10-28
2000-10-28
2000-10-28
2000-10-28
2000-10-28
2000-10-28
2000-10-28
2000-10-29
2000-10-29
2000-11-04
2000-11-04
2000-11-04
2000-11-04

...

2010-04-17
2010-04-17
2010-04-17
2010-04-17
2010-04-17
2010-04-17
2010-04-17
2010-04-18
2010-04-18
2010-04-19
2010-04-21
2010-04-24
2010-04-24
2010-04-24
2010-04-24
2010-04-24
2010-04-24
2010-04-25
2010-04-25
2010-04-25
2010-04-25
2010-05-01
2010-05-01
2010-05-01
2010-05-01
2010-05-01
2010-05-02
2010-05-02
2010-05-02
2010-05-03
2010-05-03
2010-05-05
2010-05-05
2010-05-09
2010-05-09
2010-05-09
2010-05-09
2010-05-09
2010-05-09
2010-05-09
2010-05-09
2010-05-09
2010-05-09
2010-08-14
2010-08-14
2010-08-14
2010-08-14
2010-08-14
2010-08-14
2010-08-14
2010-08-14
2010-08-15
2010-08-16
2010-08-21
2010-08-21
2010-08-21
2010-08-21
2010-08-21
2010-08-21
2010-08-21
2010-08-22
2010-08-22
2010-08-23
2010-08-28
2010-08-28
2010-08-28
2010-08-28
2010-08-28
2010-08-28
2010-08-29
2010-08-29
2010-08-29
2010-08-29
2010-09-11
2010-09-11
2010-09-11
2010-09-11
2010-09-11
2010-09-11
2010-09-11
2010-09-11
2010-09-12
2010-09-13
2010-09-18
2010-09-18
2010-09-18
2010-09-18
2010-09-18
2010-09-18
2010-09-18
2010-09-19

2012-04-10
2012-04-11
2012-04-11
2012-04-11
2012-04-11
2012-04-14
2012-04-14
2012-04-14
2012-04-14
2012-04-15
2012-04-16
2012-04-21
2012-04-21
2012-04-21
2012-04-21
2012-04-21
2012-04-21
2012-04-21
2012-04-22
2012-04-22
2012-04-22
2012-04-24
2012-04-28
2012-04-28
2012-04-28
2012-04-28
2012-04-28
2012-04-28
2012-04-28
2012-04-29
2012-04-29
2012-04-30
2012-05-01
2012-05-01
2012-05-02
2012-05-02
2012-05-05
2012-05-06
2012-05-06
2012-05-06
2012-05-06
2012-05-06
2012-05-06
2012-05-06
2012-05-07
2012-05-08
2012-05-13
2012-05-13
2012-05-13
2012-05-13
2012-05-13
2012-05-13
2012-05-13
2012-05-13
2012-05-13
2012-05-13
2012-08-18
2012-08-18
2012-08-18
2012-08-18
2012-08-18
2012-08-18
2012-08-18
2012-08-19
2012-08-19
2012-08-20
2012-08-22
2012-08-25
2012-08-25
2012-08-25
2012-08-25
2012-08-25
2012-08-25
2012-08-25
2012-08-26
2012-08-26
2012-09-01
2012-09-01
2012-09-01
2012-09-01
2012-09-01
2012-09-01
2012-09-02
2012-09-02
2012-09-02
2012-09-15
2012-09-15
2012-09-15
2012-09-15
2012-09-15
2012-09-15

2014-03-29
2014-03-29
2014-03-29
2014-03-30
2014-03-30
2014-03-31
2014-04-05
2014-04-05
2014-04-05
2014-04-05
2014-04-05
2014-04-05
2014-04-05
2014-04-06
2014-04-06
2014-04-07
2014-04-12
2014-04-12
2014-04-12
2014-04-12
2014-04-12
2014-04-12
2014-04-13
2014-04-13
2014-04-15
2014-04-16
2014-04-16
2014-04-19
2014-04-19
2014-04-19
2014-04-19
2014-04-19
2014-04-19
2014-04-20
2014-04-20
2014-04-20
2014-04-21
2014-04-26
2014-04-26
2014-04-26
2014-04-26
2014-04-26
2014-04-26
2014-04-27
2014-04-27
2014-04-27
2014-04-28
2014-05-03
2014-05-03
2014-05-03
2014-05-03
2014-05-03
2014-05-03
2014-05-03
2014-05-04
2014-05-04
2014-05-05
2014-05-06
2014-05-07
2014-05-07
2014-05-11
2014-05-11
2014-05-11
2014-05-11
2014-05-11
2014-05-11
2014-05-11
2014-05-11
2014-05-11
2014-05-11
2014-08-16
2014-08-16
2014-08-16
2014-08-16
2014-08-16
2014-08-16
2014-08-16
2014-08-17
2014-08-17
2014-08-18
2014-08-23
2014-08-23
2014-08-23
2014-08-23
2014-08-23
2014-08-23
2014-08-24
2014-08-24
2014-08-24
2014-08-25
2014-08-30

2016-03-19
2016-03-19
2016-03-19
2016-03-19
2016-03-20
2016-03-20
2016-03-20
2016-03-20
2016-04-02
2016-04-02
2016-04-02
2016-04-02
2016-04-02
2016-04-02
2016-04-02
2016-04-02
2016-04-03
2016-04-03
2016-04-09
2016-04-09
2016-04-09
2016-04-09
2016-04-09
2016-04-09
2016-04-09
2016-04-10
2016-04-10
2016-04-10
2016-04-13
2016-04-16
2016-04-16
2016-04-16
2016-04-16
2016-04-16
2016-04-16
2016-04-17
2016-04-17
2016-04-17
2016-04-18
2016-04-19
2016-04-20
2016-04-20
2016-04-20
2016-04-21
2016-04-23
2016-04-23
2016-04-23
2016-04-23
2016-04-24
2016-04-24
2016-04-25
2016-04-30
2016-04-30
2016-04-30
2016-04-30
2016-04-30
2016-04-30
2016-05-01
2016-05-01
2016-05-01
2016-05-02
2016-05-07
2016-05-07
2016-05-07
2016-05-07
2016-05-07
2016-05-07
2016-05-07
2016-05-08
2016-05-08
2016-05-08
2016-05-10
2016-05-11
2016-05-11
2016-05-11
2016-05-15
2016-05-15
2016-05-15
2016-05-15
2016-05-15
2016-05-15
2016-05-15
2016-05-15
2016-05-15
2016-05-17
2016-08-13
2016-08-13
2016-08-13
2016-08-13
2016-08-13
2016-08-13

2018-03-03
2018-03-03
2018-03-03
2018-03-03
2018-03-04
2018-03-04
2018-03-05
2018-03-10
2018-03-10
2018-03-10
2018-03-10
2018-03-10
2018-03-10
2018-03-10
2018-03-11
2018-03-11
2018-03-12
2018-03-17
2018-03-17
2018-03-17
2018-03-17
2018-03-31
2018-03-31
2018-03-31
2018-03-31
2018-03-31
2018-03-31
2018-03-31
2018-03-31
2018-04-01
2018-04-01
2018-04-07
2018-04-07
2018-04-07
2018-04-07
2018-04-07
2018-04-07
2018-04-07
2018-04-07
2018-04-08
2018-04-08
2018-04-14
2018-04-14
2018-04-14
2018-04-14
2018-04-14
2018-04-14
2018-04-14
2018-04-15
2018-04-15
2018-04-16
2018-04-17
2018-04-18
2018-04-19
2018-04-19
2018-04-21
2018-04-21
2018-04-22
2018-04-22
2018-04-22
2018-04-23
2018-04-28
2018-04-28
2018-04-28
2018-04-28
2018-04-28
2018-04-28
2018-04-28
2018-04-29
2018-04-29
2018-04-30
2018-05-04
2018-05-05
2018-05-05
2018-05-05
2018-05-05
2018-05-05
2018-05-05
2018-05-06
2018-05-06
2018-05-06
2018-05-08
2018-05-09
2018-05-09
2018-05-09
2018-05-09
2018-05-10
2018-05-13
2018-05-13
2018-05-13
2018-05-13

In [6]:
df.head(1000)[900:1000]

,season,date,home,score,away,hPos,hGSPG,hGCPG,hCSPG,hPtsPG,...,hPTSPG7,hGD3,hGD5,hGD7,aPTSPG3,aPTSPG5,aPTSPG7,aGD3,aGD5,aGD7
900,11,2002-11-23,bolton,1-1,chelsea,18.0,1.23,1.85,0.15,0.92,...,0.857,0.000,-0.4,-0.571,2.333,2.6,1.857,1.333,1.8,1.000
901,11,2002-11-23,everton,1-0,west brom,4.0,1.21,1.07,0.43,1.86,...,2.571,1.000,1.0,0.571,0.667,0.6,0.429,-0.333,-0.6,-0.857
902,11,2002-11-23,fulham,3-2,liverpool,13.0,1.21,1.29,0.29,1.14,...,0.714,-1.000,-1.2,-0.714,1.333,2.0,2.286,0.333,0.6,1.000
903,11,2002-11-23,man utd,5-3,newcastle,5.0,1.29,0.93,0.29,1.64,...,1.714,-0.333,-0.2,0.571,2.000,1.8,2.143,0.667,0.0,0.429
904,11,2002-11-23,middlesbrough,3-1,man city,7.0,1.14,0.79,0.50,1.50,...,1.429,-0.667,-0.6,0.286,2.000,1.8,1.286,0.667,0.2,-0.571
905,11,2002-11-23,southampton,3-2,arsenal,11.0,1.00,1.07,0.36,1.21,...,1.571,-0.667,0.2,0.429,3.000,1.8,2.143,1.667,0.6,1.143
906,11,2002-11-23,sunderland,0-1,birmingham,17.0,0.57,1.14,0.36,1.00,...,1.286,0.667,0.2,0.000,1.333,1.0,1.143,-0.333,-0.6,-0.571
907,11,2002-11-24,charlton,3-1,blackburn,16.0,0.79,1.29,0.29,1.00,...,1.000,0.000,0.0,-0.429,0.667,1.6,1.571,-0.333,0.6,0.571
908,11,2002-11-24,spurs,2-0,leeds,10.0,1.21,1.50,0.21,1.43,...,1.000,-1.667,-0.8,-0.857,1.000,0.8,0.714,-0.667,-0.6,-0.857
909,11,2002-11-30,arsenal,3-1,aston villa,1.0,2.20,1.07,0.33,2.13,...,1.714,1.000,0.6,0.571,2.333,1.8,1.429,1.667,1.0,0.571


In [7]:
df.to_csv("main_features_v2.csv",index=False)